In [1]:
import os
os.environ["HADOOP_USER_NAME"] = "hadoop"

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

In [3]:
sc = SparkContext("spark://bigdata.n12hiobdl5pevfjsrnvhrls1ed.jx.internal.cloudapp.net:7077")
sparkSession = (
    SparkSession.builder.appName("example-pyspark-read-and-write")
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
    .getOrCreate()
)

In [4]:
# Read from hdfs
opusdata = sparkSession.read.csv(
    "hdfs://hadoop-namenode:8020/prova/opusdata.csv", header=True, inferSchema=True
)
opusdata.show()

# sc = spark.sparkContext

+--------------------+---------------+----------+-----------------+-------------------+------------------------+---------+--------------------+--------------------+-----------------+-----------------+------+------------+
|          movie_name|production_year|movie_odid|production_budget|domestic_box_office|international_box_office|   rating|       creative_type|              source|production_method|            genre|sequel|running_time|
+--------------------+---------------+----------+-----------------+-------------------+------------------------+---------+--------------------+--------------------+-----------------+-----------------+------+------------+
|Madea's Family Re...|           2006|   8220100|         10000000|           63257940|                   62581|    PG-13|Contemporary Fiction|       Based on Play|      Live Action|           Comedy|     1|        null|
|              Krrish|           2006|  58540100|         10000000|            1430721|                31000000|Not 

In [5]:
opusdata_filter_0 = opusdata.filter(opusdata["production_budget"] != 0)

In [6]:
opusdata_filter_0 = opusdata_filter_0.filter(
    opusdata_filter_0["domestic_box_office"] != 0
)

In [7]:
opusdata_filter_0 = opusdata_filter_0.filter(
    opusdata_filter_0["international_box_office"] != 0
)

In [8]:
opusdata_dropped = opusdata_filter_0.drop(
    "movie_odid", "running_time", "production_method", "creative_type", "source"
)

In [9]:
opusdata_years = opusdata_dropped.filter(opusdata_dropped["production_year"] >= 2010)

In [10]:
opusdata_distinct = opusdata_years.dropDuplicates(["movie_name", "production_year"])

In [11]:
opusdata_total_box_office = opusdata_distinct.withColumn(
    "total_box_office",
    opusdata_distinct["domestic_box_office"]
    + opusdata_distinct["international_box_office"],
).drop("domestic_box_office", "international_box_office")

In [12]:
opusdata_droppped_na = opusdata_total_box_office.na.drop(subset=["genre", "sequel"])

In [13]:
opusdata_droppped_na.coalesce(1).write.mode("overwrite").option('header',True).csv(
    "hdfs://hadoop-namenode:8020/processed/opusdata_00.csv"
)